In [77]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [78]:
data = pd.read_csv('data/anpc_final.csv')

C:\Users\tiago\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573971 entries, 0 to 573970
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Numero                                  573971 non-null  object 
 1   DataOcorrencia                          573966 non-null  object 
 2   DataFechoOperacional                    418785 non-null  object 
 3   Natureza                                573966 non-null  object 
 4   EstadoOcorrencia                        573966 non-null  object 
 5   Distrito                                573966 non-null  object 
 6   Concelho                                573966 non-null  object 
 7   Freguesia                               573964 non-null  object 
 8   Localidade                              573964 non-null  object 
 9   Latitude                                573962 non-null  object 
 10  Longitude                               5739

In [80]:
data.describe()

,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
count,573962.000000,573962.000000,573962.000000,573962.000000
mean,1.522423,4.114807,0.018506,0.025385
std,2.303219,8.293575,0.220632,0.351961
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000
75%,1.000000,4.000000,0.000000,0.000000
max,354.000000,2170.000000,41.000000,61.000000


In [81]:
data[1:3]

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
1,2016010098470,10/01/2016 00:30:00,NaN,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0
2,2016020002249,24/01/2016 18:57:00,24/01/2016 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0


In [82]:
#Número de ocorencias por distrito

data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
0                       2
dtype: int64

In [83]:
data[data['Distrito'] == '0']

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
83938,LAR SAO JOSE,"39,5983240486353","-8,39819137470253",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262081,Travessa da Água da Flor - Frent PSP,"38,71853771","-9,165754765",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
#Retirar ocorrências sem distrito definido
data.drop(data[(data['Distrito'] == '0')].index, inplace=True)

In [85]:
data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
dtype: int64

In [86]:
##HISTOGRAMA NÚMERO DE OCORRêNCIAS POR DISTRITO## 

In [92]:
data['DataOcorrencia']=pd.to_datetime(data['DataOcorrencia'])
data['DataOcorrencia'].dt.year.unique()

array([2016.,   nan, 2017., 2018.])

Dados entre os anos 2016 e 2018 - comparar com populção por distrito 

In [109]:
pd.read_excel('data/pop.xlsx')

,Nome,Divisão Administrativa,População Residente,População/km2,Ano*
0,Lisboa,Distrito Administrativo,2275591,824.2,2021
1,Porto,Distrito Administrativo,1786656,746.0,2021
2,Setúbal,Distrito Administrativo,875656,172.9,2021
3,Braga,Distrito Administrativo,846515,312.8,2021
4,Aveiro,Distrito Administrativo,700964,250.5,2021
5,Faro,Distrito Administrativo,467495,94.6,2021
6,Leiria,Distrito Administrativo,458679,130.9,2021
7,Santarém,Distrito Administrativo,425431,67.9,2021
8,Coimbra,Distrito Administrativo,408631,103.5,2021
9,Viseu,Distrito Administrativo,351592,70.2,2021
